In [1]:
from __future__ import print_function
import tweepy
import json
from pymongo import MongoClient
import numpy as np
import pandas as pd
from datetime import datetime
import time
from datetime import timedelta
from kafka import KafkaConsumer
import requests

## In questo File si eseguono tutte le operazioni dedicate al Consumer:

## Definisco dove salvare ogni oggetto in arrivo dal topic

In [2]:
MONGO_HOST= 'mongodb://localhost/test' 

In [3]:
def troncahashtag (lista):
    for i in range(0, len(lista)):
        lista[i] = lista[i][1:]
    return lista

In [4]:
def minuscolo(hashtags):
    for i in range(0,len(hashtags)):
        hashtags[i] = hashtags[i].lower()
    return (hashtags)

In [5]:
Match = minuscolo(["#WeTheNorth", "#Raptors", "#TorontoRaptors", "#NBAFinals", "#NBAPlayoffs", "#RTZ",
                   "#DubNation", "#WarriorsGround", "#StrengthInNumbers", "#GoldenStateWarriors", "#GswvsTor", "#TorvsGsw" ])
Away = minuscolo(["#WeTheNorth", "#Raptors", "#TorontoRaptors", "#RTZ"])
Home = minuscolo(["#DubNation", "#WarriorsGround", "#StrengthInNumbers", "#GoldenStateWarriors"])
N = minuscolo(["#NBAFinals", "#NBAPlayoffs", "#GswvsTor", "#TorvsGsw"])


Home2 = troncahashtag(Home)
Away2 = troncahashtag(Away)
client = MongoClient(MONGO_HOST)
db = client.test

## Funzione per indirizzare al salvataggio l'oggetto appena arrivato dal topic

In [374]:
def Filter(tweetobj, data):
    if (bool(set(tweetobj["hashtag"]) & set(Home2)) and  bool(set(tweetobj["hashtag"]) & set(Away2))):
        print("\n Neutro")
        with open("Neutro.json", "a") as neutro:
            json.dump(final, neutro, indent = 1)
            neutro.write("\n \n \n \n")
        print("Il tweet è stato correttamente inserito nel file Neutro.json \n")
        #Inserimento nella Collection
        db.Neutro.insert_one(final) 
        with open("Neutro_tweet.json", "a") as neutro_tweet:
            json.dump(data, neutro_tweet, indent = 1)
            neutro_tweet.write("\n \n \n \n")
        print("Il tweet è stato correttamente inserito nella collection Neutro\n")
    elif bool(set(tweetobj["hashtag"]) & set(Home2)) and not bool(set(tweetobj["hashtag"]) & set(Away2)):
        #Inserimento del tweet nel File e nella Mongo Collection
        #Inserimento nel File
        print("\n Team 1")
        with open("Team1.json", "a") as team1:
            json.dump(final, team1, indent = 1)
            team1.write("\n \n \n \n")
        print("Il tweet è stato correttamente inserito nel file Team1.json \n")
        #Inserimento nella Collection
        db.GoldenState.insert_one(final)
        with open("Team1_tweet.json", "a") as team1_tweet:
            json.dump(data, team1_tweet, indent = 1)
            team1_tweet.write("\n \n \n \n")        
        print("Il tweet è stato correttamente inserito nella collection Team1 \n")
    elif not bool(set(tweetobj["hashtag"]) & set(Home2)) and bool(set(tweetobj["hashtag"]) & set(Away2)):
        #Inserimento del tweet nel File e nella Mongo Collection
        #Inserimento nel File
        print("\n Team 2")
        with open("Team2.json", "a") as team2:
            json.dump(final, team2, indent = 1)
            team2.write("\n \n \n \n")
        print("Il tweet è stato correttamente inserito nel file Team2.json \n")
        #Inserimento nella Collection
        db.Toronto.insert_one(final)
        with open("Team2_tweet.json", "a") as team2_tweet:
            json.dump(data, team2_tweet, indent = 1)
            team2_tweet.write("\n \n \n \n")        
        print("Il tweet è stato correttamente inserito nella collection Team2 \n")
    else:
        with open("Neutro.json", "a") as neutro:
            print("\n Neutro")
            json.dump(final, neutro, indent = 1)
            neutro.write("\n \n \n \n")
            print("Il tweet è stato correttamente inserito nel file Neutro.json \n")
            #Inserimento nella Collection
            db.Neutro.insert_one(final)
        with open("Neutro_tweet.json", "a") as neutro_tweet:
            json.dump(data, neutro_tweet, indent = 1)
            neutro_tweet.write("\n \n \n \n")            
        print("Il tweet è stato correttamente inserito nella collection Neutro\n")

## Creazione del consumer e connessione con il Topic:

In [375]:
consumer = KafkaConsumer(bootstrap_servers='sandbox-hdp.hortonworks.com:6667', value_deserializer=lambda m: json.loads(m.decode('utf-8')))
consumer.subscribe(['gara1'])

## Lasciamo commentata

i = 0
while(1):
    r = requests.get(url = URL, params = PARAMS)                                                 # richiesta
    statistiche = r.json()
    if len(statistiche) > 1:
        statistiche = stats_pulite(statistiche)
        with open("/home/maria_dev/dati_NBA/tanti_file/integrazione/data" + str(i) + ".json", 'w') as outfile:
            json.dump(statistiche, outfile, indent = 1)
            i = i+1
        time.sleep(30)

## Ciclo che permette di ottenere il messaggio dal Producer via Topic e replicarlo nel database:

In [833]:
while(1):
    for message in consumer:
        messaggio = message[6]
        #print(message)
        #print(messaggio)
        
        tweetFiltered = messaggio[0]
        #print(messaggio[0])
        #print(tweetFiltered)
        
        data = messaggio[1]
        richiesta = messaggio[2]
        #print(tweetFiltered)
        #print(data)
           
        #twitterFiltered = messaggio[0]
        #data = messaggio[1]
        
        #print("\n \n \n \n Ho ricevuto un messaggio e lo stampo: \n \n \n \n")
        #print(tweetFiltered)
        #print(data)
        print("\n\n\n Il risultato dell'integrazione è il seguente") #4:48
        final = tweetFiltered
        Filter(final, data)
        print(final)
#        time.sleep(30)




 Il risultato dell'integrazione è il seguente

 Neutro
Il tweet è stato correttamente inserito nel file Neutro.json 

Il tweet è stato correttamente inserito nella collection Neutro

{'_id': '1139394021694922752', 'n_tweet': 1438003, 'time_tweet': '04:48:03', 'date_tweet': '2019-06-14', 'source': '<a href="http://twitter.com/download/iphone" rel="nofollow">Twitter for iPhone</a>', 'retweet_count': 0, 'placeTweet': None, 'placeUser': '717 x 412', 'stats': {'event_date': '2019-06-14', 'event_key': '42208', 'time': '03:49:19', 'event_status': 'Finished', 'final_score': '110 - 114', 'scores': {'1stQuarter': [{'score_home': '32', 'score_away': '33'}], '2ndQuarter': [{'score_home': '25', 'score_away': '27'}], '3rdQuarter': [{'score_home': '31', 'score_away': '26'}], '4thQuarter': [{'score_home': '22', 'score_away': '28'}]}, 'statistics': {'Match': [{'type': 'Field Goals Attempted', 'home': '79', 'away': '82'}, {'type': 'Field Goals Made', 'home': '39', 'away': '39'}, {'type': 'Field Goals

DuplicateKeyError: E11000 duplicate key error collection: test.Toronto index: _id_ dup key: { : "1139394017433538560" }